In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [40]:
date_dimension = pd.read_csv("date_dimension.csv")
weather_dimension = pd.read_csv("weather_dimension.csv")
mobility_dimension = pd.read_csv("Mobility_dimension.csv")
patient_dimension = pd.read_csv("patient_dimension.csv")
phu_location_dimension = pd.read_csv("phu_location_dimension.csv")
special_measure_dimension = pd.read_csv("special_measures_dimension.csv")

In [3]:
patients = pd.read_csv("patients.csv")

In [4]:
patients = pd.read_csv("patients.csv")
s_date = datetime.strptime('20200901', '%Y%m%d')
e_date = datetime.strptime('20201231', '%Y%m%d')
patients = patients[(pd.to_datetime(patients['Case_Reported_Date']) >= s_date) & (pd.to_datetime(patients['Case_Reported_Date']) <= e_date)]
patients = patients[patients['Reporting_PHU'].str.contains("Peel") | patients['Reporting_PHU'].str.contains("York") | patients['Reporting_PHU'].str.contains("Durham") | patients['Reporting_PHU'].str.contains("Halton") | patients['Reporting_PHU_City'].str.contains("Toronto") | patients['Reporting_PHU_City'].str.contains("Ottawa")]


In [98]:
patients["Accurate_Episode_Date"] = pd.to_datetime(patients["Accurate_Episode_Date"] ).dt.strftime('%Y-%m-%d')
patients["Case_Reported_Date"] = pd.to_datetime(patients["Case_Reported_Date"] ).dt.strftime('%Y-%m-%d')
patients["Test_Reported_Date"] = pd.to_datetime(patients["Test_Reported_Date"] ).dt.strftime('%Y-%m-%d')
patients["Specimen_Date"] = pd.to_datetime(patients["Specimen_Date"] ).dt.strftime('%Y-%m-%d')

In [15]:
pd.set_option('display.max_rows', None)

In [18]:
patients.head(900)

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
418,419,2020-12-16,2020-12-17,2020-12-17,2020-12-16,80s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
419,420,2020-12-15,2020-12-17,2020-12-17,2020-12-15,<20,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
421,422,2020-12-11,2020-12-17,2020-12-17,2020-12-16,20s,MALE,NO KNOWN EPI LINK,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
422,423,2020-12-10,2020-12-17,2020-12-17,2020-12-15,30s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
424,425,2020-12-15,2020-12-17,2020-12-17,2020-12-16,30s,FEMALE,CC,Resolved,NaN,2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
425,426,2020-12-10,2020-12-15,2020-12-15,2020-12-11,70s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
428,429,2020-12-13,2020-12-17,2020-12-17,2020-12-15,20s,MALE,NO KNOWN EPI LINK,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
429,430,2020-12-09,2020-12-17,2020-12-17,2020-12-15,20s,MALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893
434,435,2020-12-08,2020-12-17,2020-12-17,2020-12-16,50s,FEMALE,CC,Resolved,NaN,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
439,440,2020-12-17,2020-12-17,2020-12-17,2020-12-17,30s,FEMALE,CC,Resolved,NaN,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,www.peelregion.ca/health/,43.647471,-79.708893


In [28]:
fact_table =  pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key', 'Specimen_date_key',
                                    'Patient_key',"PHU_Location_key","Mobility_key",
                                   'Special_measure_key', 'Weather_key', 'isResolved', 'isUnresolved', 'isFatal'])


In [41]:
mobility_dimension[(mobility_dimension["Province"]=='ON') & (mobility_dimension["Subregion_2"]==patients["Reporting_PHU"].split(" ")[0])]



AttributeError: 'Series' object has no attribute 'split'

In [29]:

for idx, row in patients.iterrows():
    
    if idx%1000 ==0:
         print(idx, len(fact_table),end="\r")
    
    #print(date_dimension[date_dimension["date"]==row["Accurate_Episode_Date"]]["date_id"].values[0])
    if(len(date_dimension[date_dimension["date"]==row["Accurate_Episode_Date"]])==0):
        fact_row = [-1]
    else:
        fact_row = [date_dimension[date_dimension["date"]==row["Accurate_Episode_Date"]]["date_id"].values[0]]
    
    fact_row.append(date_dimension[date_dimension["date"]==row["Case_Reported_Date"]]["date_id"].values[0])
    
    if(len(date_dimension[date_dimension["date"]==row["Test_Reported_Date"]])==0):
        fact_row.append(-1)
    else:
        fact_row.append(date_dimension[date_dimension["date"]==row["Test_Reported_Date"]]["date_id"].values[0])
    
        
    if(len(date_dimension[date_dimension["date"]==row["Specimen_Date"]])==0):
        fact_row.append(-1)
    else:
        fact_row.append(date_dimension[date_dimension["date"]==row["Specimen_Date"]]["date_id"].values[0])
    fact_row.append(patient_dimension[patient_dimension["Row_ID"]==row["Row_ID"]]["Row_ID"].values[0])
    fact_row.append(phu_location_dimension[phu_location_dimension["PHU_location_key"]==row["Reporting_PHU_ID"]]["PHU_location_key"].values[0])
    fact_row.append(mobility_dimension[(mobility_dimension["Date"]==row["Case_Reported_Date"]) & (mobility_dimension["Subregion_2"]==row["Reporting_PHU"].split(" ")[0])]["mobility_id"].values[0])
    if (len(special_measure_dimension[(special_measure_dimension["Reporting_PHU"]==row["Reporting_PHU"]) & (special_measure_dimension["start_date"]<=row["Case_Reported_Date"]) & (special_measure_dimension["end_date"]>=row["Case_Reported_Date"])]) == 0):
        fact_row.append(-1)
    else:
        fact_row.append(special_measure_dimension[(special_measure_dimension["Reporting_PHU"]==row["Reporting_PHU"]) & (special_measure_dimension["start_date"]<=row["Case_Reported_Date"]) & (special_measure_dimension["end_date"]>=row["Case_Reported_Date"])]["measure_id"].values[0])
    fact_row.append(weather_dimension[(weather_dimension["Date"]==row["Case_Reported_Date"]) & (weather_dimension["City"]==row["Reporting_PHU"].split(" ")[0])]["Weather_ID"].values[0])
    
    
    fact_row += [ 1 if row["Outcome1"] == "Resolved" else 0 ,
                        1 if row["Outcome1"] == "Fatal" else 0,
                         1 if row["Outcome1"] == "Fatal" else 0,
                ]
    
    fact_table.loc[len(fact_table)] = fact_row

In [30]:
fact_table.to_csv("look_up_table.csv",index=False)

In [24]:
fact_table.head(50)

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_Location_key,Mobility_key,Special_measure_key,Weather_key,isResolved,isUnresolved,isFatal
0,10942,10943,10943,10942,419,2253,51576,99,2400697,1,0,0
1,10941,10943,10943,10941,420,2253,51576,99,2400697,1,0,0
2,10937,10943,10943,10942,422,2253,51576,99,2400697,1,0,0
3,10936,10943,10943,10941,423,2253,51576,99,2400697,1,0,0
4,10941,10943,10943,10942,425,2236,50806,41,6155750,1,0,0
5,10936,10941,10941,10937,426,2253,51574,99,2400697,1,0,0
6,10939,10943,10943,10941,429,2253,51576,99,2400697,1,0,0
7,10935,10943,10943,10941,430,2253,51576,99,2400697,1,0,0
8,10934,10943,10943,10942,435,2270,51961,168,4019073,1,0,0
9,10943,10943,10943,10943,440,2253,51576,99,2400697,1,0,0


In [42]:
fact_table_final =  pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key', 'Specimen_date_key','patient_key','PHU_location_key',
                                         'Mobility_key', 'Special_measure_key', 'Weather_key', 'isResolved', 'isUnresolved', 'isFatal'])





In [43]:

for idx, row in fact_table.iterrows():
    
    if idx%1000 ==0:
         print(idx, len(fact_table_final),end="\r")
    
    if(row["Onset_date_key"]==-1):
        fact_row = [' ']
    else:
        fact_row = [date_dimension[date_dimension["date_id"]==row["Onset_date_key"]]["surrogate_key"].values[0]]
    
    fact_row.append(date_dimension[date_dimension["date_id"]==row["Reported_date_key"]]["surrogate_key"].values[0])
    
    if(row["Test_date_key"]==-1):
        fact_row.append(' ')
    else:
        fact_row.append(date_dimension[date_dimension["date_id"]==row["Test_date_key"]]["surrogate_key"].values[0])
    
    if(row["Specimen_date_key"]==-1):
        fact_row.append(' ')
    else:
        fact_row.append(date_dimension[date_dimension["date_id"]==row["Specimen_date_key"]]["surrogate_key"].values[0])

    fact_row.append(patient_dimension[patient_dimension["Row_ID"]==row["Patient_key"]]["surrogate_key"].values[0])
    fact_row.append(phu_location_dimension[phu_location_dimension["PHU_location_key"]==row["PHU_Location_key"]]["surrogate_key"].values[0])
    fact_row.append(mobility_dimension[mobility_dimension["mobility_id"]==row["Mobility_key"]]["surrogate_key"].values[0])
    
    if(row["Special_measure_key"]==-1):
        fact_row.append(' ')
    else:
        fact_row.append(special_measure_dimension[special_measure_dimension["measure_id"]==row["Special_measure_key"]]["surrogate_key"].values[0])
        
    fact_row.append(weather_dimension[weather_dimension["Weather_ID"]==row["Weather_key"]]["surrogate_key"].values[0])
    fact_row.append(row['isResolved'])
    fact_row.append(row['isUnresolved'])
    fact_row.append(row['isFatal'])

    
    fact_table_final.loc[len(fact_table_final)] = fact_row

In [46]:
fact_table_final.head(600)

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,patient_key,PHU_location_key,Mobility_key,Special_measure_key,Weather_key,isResolved,isUnresolved,isFatal
0,106,107,107,106,0,0,473,15,366,1,0,0
1,105,107,107,105,1,0,473,15,366,1,0,0
2,101,107,107,106,2,0,473,15,366,1,0,0
3,100,107,107,105,3,0,473,15,366,1,0,0
4,105,107,107,106,4,1,351,6,244,1,0,0
5,100,105,105,101,5,0,471,15,366,1,0,0
6,103,107,107,105,6,0,473,15,366,1,0,0
7,99,107,107,105,7,0,473,15,366,1,0,0
8,98,107,107,106,8,2,595,25,488,1,0,0
9,107,107,107,107,9,0,473,15,366,1,0,0


In [45]:
fact_table_final.to_csv("fact_table.csv",index=False)